In [37]:
#hide
#default_exp core
#all_slow

# Tutorial: Creating tweetrel
> and an overview of its API

In [2]:
#export
from fastcore.utils import *

In [3]:
#hide
from nbdev.showdoc import *

In order to create this library, we will start out by following the initial steps of the [nbdev tutorial](https://nbdev.fast.ai/tutorial.html) to create a basic project structure. We also need to install [ghapi](https://ghapi.fast.ai/), by following the instructions on that link.

Once we have a repo cloned, based on the `nbdev_template` template, we run in our terminal:

```bash
gh-create-workflow tweet release --contexts secrets
```

Note that we add `--contexts secrets` because we'll need access to the [secrets context](https://docs.github.com/en/free-pro-team@latest/actions/reference/encrypted-secrets#using-encrypted-secrets-in-a-workflow), which we'll be using to store our Twitter API keys.

The basic workflow skeleton that this creates is:

In [2]:
wf_path  = Path('.github/workflows')
scr_path = Path('.github/scripts')
wf_path.ls(),scr_path.ls()

((#1) [Path('.github/workflows/tweet-release.yml')],
 (#1) [Path('.github/scripts/build-tweet-release.py')])

## Setting up Twitter authentication

In order to send tweets, we'll send to use the Twitter API -- we'll use the `tweepy` library for this:

In [41]:
#exports
import tweepy

The Twitter part of this tutorial isn't the main thing we want to explain, but the thing to note carefully here is how to access *secrets* in your workflow.

The Twitter API requires authentication, so we'll store our details in a GitHub secret. We'll just use one secret, we our keys stored space delimited. We generally use an [organization secret](https://docs.github.com/en/free-pro-team@latest/actions/reference/encrypted-secrets#creating-encrypted-secrets-for-an-organization), so that we can update secrets in one place as needed.

Contexts are passed to ghapi as json-encoded environment variables. Each variable name starts with `CONTEXT_` -- so for instance the `secrets` context is `CONTEXT_SECRETS`. In order to simulate this when testing, ensure that you have set an appropriate environment variable before importing `ghapi`. We've put our JSON encoded secrets for testing into a `.secrets` file (which we added to `.gitignore` so it wouldn't be pushed to our repo), so we'll use it here.

In [4]:
os.environ['CONTEXT_SECRETS'] = Path('.secrets').read_text()

Now we can import `ghapi`.

In [42]:
#exports
from ghapi import *

If you now check the `context_secrets` variable, you should find your secrets available as an `AttrDict`:

In [6]:
assert 'TWITTER' in context_secrets

Now we'll create a function to unpack our keys, tokens, and secrets for Twitter auth, and login to the API:

In [38]:
#exports
def twitter_api():
    consumer_key,consumer_secret,access_token,access_token_secret = context_secrets.TWITTER.split()
    auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
    auth.set_access_token(access_token,access_token_secret)
    return tweepy.API(auth)

We can test our login by creating and deleting a tweet:

```python
import time
twapi = twitter_api()
stat = twapi.update_status("Please ignore - testing API")
time.sleep(1)
twapi.destroy_status(stat.id)
```

## Responding to the `release` event

We can get a sample `release` event payload as follows:

In [8]:
example = example_payload(Event.release)
list(example), example.action

(['action', 'release', 'repository', 'sender'], 'published')

The `release` section has the following keys:

In [9]:
', '.join(example.release)

'url, assets_url, upload_url, html_url, id, node_id, tag_name, target_commitish, name, draft, author, prerelease, created_at, published_at, assets, tarball_url, zipball_url, body'

We can create a function that formats a tweet based on this information:

In [39]:
#exports
def tweet_text(payload):
    rel = payload.release
    owner,repo = re.findall(r'https://api.github.com/repos/([^/]+)/([^/]+)/', rel.url)[0]
    tweet_tmpl = "New #{repo} release: v{tag_name}. {html_url}\n\n{body}"
    res = tweet_tmpl.format(repo=repo, tag_name=rel.tag_name, html_url=rel.html_url, body=rel.body)
    if len(res)<=280: return res
    return res[:279] + "…"

...and test it:

In [11]:
print(tweet_text(example))

New #Hello-World release: v0.0.1. https://github.com/Codertocat/Hello-World/releases/tag/0.0.1

None


The sample release payload from GitHub happens to have an empty `body`, but other than that, this looks good.

That's all we need to create our function:

In [40]:
#exports
def send_tweet():
    payload = context_github.event
    if 'workflow' in payload: payload = example_payload(Event.release)
    if payload.action == 'published': return twitter_api().update_status(tweet_text(payload))

We can pop this into our python script, along with `send_tweet()`. Since we're using nbdev, we can enter the following in our terminal to do all that:

```bash
nbdev_build_lib
cp tweetrel/core.py .github/scripts/build-tweet-release.py
echo -e "\nsend_tweet()" >>  .github/scripts/build-tweet-release.py
```

After we push to GitHub, we can test it out in the same way we showed in the [GitHub Actions tutorial](https://ghapi.fast.ai/tutorial_actions.html), first by logging in and getting a reference to our workflow:

In [15]:
api = GhApi(owner='fastai', repo='tweetrel', token=github_token())
wf = api.actions.get_workflow('tweet-release.yml')

and then running it:

```python
api.actions.create_workflow_dispatch(wf.id, ref='master')
```

After you run this, you should find the workflow appears in your "Actions" tab on GitHub, and a tweet will appear in your twitter timeline.

### Some little improvements

One issue is that currently you'll see three workflows being triggered on the Actions tab in GitHub. That's because the "created" and "released" types are resulting in a trigger, as well as "published". Our function is checking which is being used, and only tweeting for "published", but the runs are still being recorded. This isn't necessarily a big problem, but if you'd like things to be cleaner, you can edit the `tweet-release.yml` file to add the following line after the `release:` line:

```bash
types: [published]
```

It's also possible to do a more complete end-to-end test by actually making a release, checking that a new run is created, and checking the result of that run:

```python
n_runs = api.actions.list_workflow_runs(wf.id).total_count
try:
    rel = api.create_release('test', body='body')
    time.sleep(30)
    runs = api.actions.list_workflow_runs(wf.id)
    test(runs.total_count, n_runs, gt)
    test_eq(runs.workflow_runs[0].conclusion, 'success')
finally: api.delete_release(rel)
```

## Distributing your new Action

The above steps are all that's needed to create a Python-based workflow for use in a single project. To make it accessible for any project (including other people's projects), we need to distribute it. The easiest way to do that is with `pip`. `pip` can install modules directly from GitHub, or from the PyPi repository.

Our first step is to add a command that makes it easy to install the `tweetrel` workflow into a project. We can use `fastcore.script.call_parse` to create a function that can be run from the terminal.

In [44]:
#exports
from fastcore.script import *

In the function, we'll use `fill_workflow_templates` instead of `gh-create-workflow` because it allows us to customize each part of the YAML workflow file exactly as we need it:

In [45]:
#exports
@call_parse
def install():
    fill_workflow_templates(
        name='tweet', event="release:\n  types: [published]",
        run=f'pip install -Uq git+https://github.com/fastai/tweetrel.git',
        context=env_contexts('secrets'),
        script="import tweetrel\ntweetrel.send_tweet()"
    )

## Export -

In [1]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
